In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import sqlite3
import lxml

In [2]:
base = 'https://www.einbuergerungstest-online.eu/'
question_subsite = 'fragen/'
pages = [''] + [str(i) for i in np.arange(2,11)] #first pages does not start with '1' but with ''

In [3]:
def get_questions(soup):
    mydivs = soup.findAll("div", {"class": "questions-question-text"})
    questions  =[[list(j.children) for j in list(i.children)] for i in mydivs]

    clean = []
    for q in questions:
        #get the link if existing - else set a link to None
        try: 
            qid, qtext = q
            qlink = None
        except:
            qid, qtext, qlink = q
            #extract the link
            qlink = qlink[0]['href']



        #extract the text without the link
        if qtext[0].format:
            rawtext = qtext[0].extract()
        else:
            rawtext = qtext[0].text

        clean.append((qid[0], rawtext, qlink))
    return clean

In [4]:
def get_answers(soup):
    answers = list(soup.findAll("li"))
    raw = [list(i.children)[0] for i in answers][35:-12] #get only the answers on the page and ignore the rest

    # clear the answers from the green coloring on the website which indicates truth
    # add truth indicator as 0 or 1 
    clean = []
    for content in raw:
        if content.format:
            clean.append((content.extract(), 0))
        else:
            clean.append((content.text, 1))   
    return np.array(clean[:30*4]).reshape(30,4,2)

In [5]:
questions = []
answers = []

for page in pages:
    url_to_scrape = base + question_subsite + page
    r = requests.get(url_to_scrape)
    soup = BeautifulSoup(r.text, "lxml")

    answers.append((get_answers(soup)))
    questions.append((get_questions(soup)))

    print(page)
    


2
3
4
5
6
7
8
9
10


In [6]:
import sqlite3
conn = sqlite3.connect('GermanCitizenship.db')
c = conn.cursor() 


c.execute("CREATE TABLE IF NOT EXISTS Question ([ID] INTEGER PRIMARY KEY,[Text] VARCHAR(1000), [Image] VARCHAR(250))")          

c.execute('''CREATE TABLE IF NOT EXISTS Answer ([ID] INTEGER PRIMARY KEY, [QuestionID] INTEGER, [Text] VARCHAR(1000),  [True] BOOL)''')

c.execute('''CREATE TABLE IF NOT EXISTS ProgressEvent ([generated_id] INTEGER PRIMARY KEY, [QuestionID] INTEGER, [Success] BOOL)''')
        
                
conn.commit()

In [7]:
# add QUESTIONS to database 
for q in np.array(questions).reshape(300,3):
    query = 'INSERT INTO Question (ID, Text, Image) VALUES ("{}", "{}", "{}");'.format(q[0], q[1], q[2])
    c.execute(query)
    
conn.commit()

# add ANSWERS to database
for qid, a in enumerate(np.array(answers).reshape(300, 4, 2)):
    for poss_answer in a:
        query = 'INSERT INTO Answer (QuestionID, Text, True) VALUES ("{}", "{}", "{}");'.format(qid+1, poss_answer[0], poss_answer[1])
        c.execute(query)
    
conn.commit()    


In [8]:
conn = sqlite3.connect('GermanCitizenship.db')
c = conn.cursor() 
c.execute('Select * from Question')

In [9]:
import pandas as pd
pd.read_sql('Select * from Question', conn)

,ID,Text,Image
0,1,In Deutschland dürfen Menschen offen etwas geg...,None
1,2,In Deutschland können Eltern bis zum 14. Leben...,None
2,3,Deutschland ist ein Rechtsstaat. Was ist damit...,None
3,4,Welches Recht gehört zu den Grundrechten in De...,None
4,5,Wahlen in Deutschland sind frei. Was bedeutet ...,None
5,6,Wie heißt die deutsche Verfassung?,None
6,7,"Welches Recht gehört zu den Grundrechten, die ...",None
7,8,Was steht nicht im Grundgesetz von Deutschland?,None
8,9,Welches Grundrecht gilt in Deutschland nur für...,None
9,10,Was ist mit dem deutschen Grundgesetz vereinbar?,None


In [10]:
import pandas as pd
pd.read_sql('Select * from Answer', conn)

,ID,QuestionID,Text,True
0,1,1,hier Religionsfreiheit gilt.,0
1,2,1,die Menschen Steuern zahlen.,0
2,3,1,die Menschen das Wahlrecht haben.,0
3,4,1,hier Meinungsfreiheit gilt.,1
4,5,2,Geschichtsunterricht teilnimmt.,0
5,6,2,Religionsunterricht teilnimmt.,1
6,7,2,Politikunterricht teilnimmt.,0
7,8,2,Sprachunterricht teilnimmt.,0
8,9,3,Alle Einwohner / Einwohnerinnen und der Staat ...,1
9,10,3,Der Staat muss sich nicht an die Gesetze halten.,0
